In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0,1))

import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, LSTM, SimpleRNN, GRU, Bidirectional, RNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from sklearn.metrics import mean_squared_error
from itertools import product


In [2]:
def remove_commas(data):
  new_data = []
  for row in data:
    row_ = []
    for elem in row:
      if ',' in str(elem):
        elem = elem.replace(',', '')
      elem = float(elem)
      row_.append(elem)
    new_data.append(row_)

  return np.array(new_data)

In [ ]:
# read in train data
data = pd.read_csv('drive/MyDrive/Colab Notebooks/Google_Stock_Price_Train.csv')
data['Date'] = pd.to_datetime(data['Date'])
colnames = list(data.columns[1:])

# divide train and val and scale them
condition = (data['Date'] > '2016-01-01') & (data['Date'] <= '2016-12-31')

data_2012_2015 = data.loc[~condition]
train_raw = remove_commas(data_2012_2015.drop(columns=['Date']).copy().values)
train_scaled = scaler.fit_transform(train_raw)

data_2016 = data.loc[condition]
val_raw = remove_commas(data_2016.drop(columns=['Date']).copy().values)
val_scaled = scaler.transform(val_raw) 

nfeatures = train_scaled.shape[1]

# read in test data and scale them
test_data = pd.read_csv('drive/MyDrive/Colab Notebooks/Google_Stock_Price_Test.csv')
test_raw = remove_commas(test_data.drop(columns=['Date']).copy().values)
test_scaled = scaler.transform(test_raw)

# Visualisation

In [ ]:
# Visualisation of Google's Stock Price

full_data_raw = np.vstack((train_raw, val_raw, test_raw))

fig, axs = plt.subplots(5,1, figsize = (10,15), sharex = True)
for i in range(len(colnames)):
  axs[i].plot(full_data_raw[:,i], color = 'blue')
  axs[i].set_title(colnames[i])
  if i == len(colnames)-1:
    axs[i].set_ylabel('Number of shares')
  else:
    axs[i].set_ylabel('Price ($)')
axs[-1].set_xlabel('Time')
#plt.tight_layout()
plt.savefig('full_data.png')
plt.show()

In [ ]:
# function to transform data into a supervised learning problem

def prepare_data(train_scaled, val_scaled, test_scaled, window_size):

  nfeatures = train_scaled.shape[0]

  X_train = []
  y_train = []

  for i in range(len(train_scaled)-window_size):
    X_train.append(train_scaled[i:i+window_size, :])
    y_train.append(train_scaled[i+window_size,:])

  X_train, y_train = np.array(X_train), np.array(y_train)

  X_val = []
  y_val = []

  full_val_set = np.vstack((train_scaled[-window_size:], val_scaled))
  for i in range(len(full_val_set)-window_size):
    X_val.append(full_val_set[i:i+window_size, :])
    y_val.append(full_val_set[i+window_size,:])

  X_val, y_val = np.array(X_val), np.array(y_val)

  # making predictions and visualise
  full_test_set = np.vstack((val_scaled[-window_size:, :], test_scaled))

  X_test = []

  for i in range(len(full_test_set)-window_size):
    X_test.append(full_test_set[i:i+window_size, :])

  X_test = np.array(X_test)

  return X_train, y_train, X_val, y_val, X_test

# Experiment 1

In [3]:
def create_model(parameters, window_size, nfeatures, model_type, bidirectional):
  units, nlayers, dropout_rate, learning_rate = parameters
  model = Sequential()

  if model_type == 'LSTM':
    for i in range(nlayers):
      if bidirectional:
        model.add(Bidirectional(LSTM(units = units, return_sequences = True, input_shape = (window_size, nfeatures))))
      else:
         model.add(LSTM(units = units, return_sequences = True, input_shape = (window_size, nfeatures)))
      model.add(Dropout(0.2))
    
    if bidirectional:
      model.add(Bidirectional(LSTM(units = units)))
    else:
      model.add(LSTM(units = units))
    model.add(Dropout(0.2)) 

  elif model_type == 'GRU':
    for i in range(nlayers):
      if bidirectional:
        model.add(Bidirectional(GRU(units = units, return_sequences = True, input_shape = (window_size, nfeatures))))
      else:
         model.add(GRU(units = units, return_sequences = True, input_shape = (window_size, nfeatures)))
      model.add(Dropout(0.2))

    if bidirectional:
      model.add(Bidirectional(GRU(units = units)))
    else:
      model.add(GRU(units = units))
    model.add(Dropout(0.2)) 

  elif model_type == 'SimpleRNN':
    for i in range(nlayers):
      if bidirectional:
        model.add(Bidirectional(SimpleRNN(units = units, return_sequences = True, input_shape = (window_size, nfeatures))))
      else:
         model.add(SimpleRNN(units = units, return_sequences = True, input_shape = (window_size, nfeatures)))
      model.add(Dropout(0.2))
    
    if bidirectional:
      model.add(Bidirectional(SimpleRNN(units = units)))
    else:
      model.add(SimpleRNN(units = units))
    model.add(Dropout(0.2)) 

  model.add(Dense(units = nfeatures))

  opt = Adam(learning_rate = learning_rate)
  model.compile(optimizer = opt, loss = 'mean_squared_error')

  return model

In [ ]:
data = []
parameters = (32, 2, 0.2, 0.01)
window_size, nfeatures = 60, train_raw.shape[1]

for model_type in ['SimpleRNN', 'GRU', 'LSTM']:
  for bidirectional in [False, True]:
    predicted_val_data = []
    predicted_test_data = []
    rmse_data = []

    for ntime in range(5):
      # run each model 5 times
      tf.keras.backend.clear_session()
      model = create_model(parameters, window_size, nfeatures, model_type, bidirectional)

      stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 50,
                                                restore_best_weights = True)

      history = model.fit(X_train, y_train, validation_data = (X_val, y_val), epochs = 100, 
                          batch_size = 32, verbose = 0, callbacks = [stop_early])
      
      predicted_val = model.predict(X_val)
      predicted_val = scaler.inverse_transform(predicted_val)

      predicted = model.predict(X_test)
      predicted = scaler.inverse_transform(predicted)

      rmse = mean_squared_error(test_raw, predicted, squared = False)
      rmse_data.append(rmse)

      predicted_val_data.append(predicted_val)
      predicted_test_data.append(predicted)
      
    mean_predicted_val = np.mean(np.dstack(predicted_val_data), axis = -1)
    mean_predicted_test = np.mean(np.dstack(predicted_test_data), axis = -1)
    mean_rmse = np.mean(rmse)

    data.append((mean_predicted_val, mean_predicted_test, mean_rmse))

    for i in range(nfeatures):
      plt.plot(val_raw[:,i], label = 'actual')
      plt.plot(mean_predicted_val[:,i], label = 'predicted')
      plt.legend()
      plt.show()

    for i in range(nfeatures):
      plt.plot(test_raw[:,i], label = 'actual')
      plt.plot(mean_predicted_test[:,i], label = 'predicted')
      plt.legend()
      plt.show()

    print('Average RMSE: ', mean_rmse)

In [ ]:
i = 0
path = 'drive/MyDrive/Colab Notebooks/results'
for name in ['SimpleRNN', 'GRU', 'LSTM']:
  for b in [False, True]:
    np.save(path + '{}_{}_predicted_val.npy'.format(name, str(b)), data[i][0])
    np.save(path +'{}_{}_predicted_test.npy'.format(name, str(b)), data[i][1])
    np.save(path +'{}_{}_mse.npy'.format(name, str(b)), data[i][2])
    i += 1

In [ ]:
path = 'drive/MyDrive/Colab Notebooks/results/'
results_dict = {'SimpleRNN_False': [],
               'SimpleRNN_True': [],
               'GRU_False': [],
               'GRU_True': [],
               'LSTM_False': [],
               'LSTM_True': []}

for key in results_dict:
  path = 'drive/MyDrive/Colab Notebooks/results/' + key + '_'
  results_dict[key] = [np.load(path+'predicted_val.npy'), np.load(path+'predicted_test.npy'), np.load(path+'mse.npy')]


In [ ]:
# checking all RMSE

all_rmse = []
for key in results_dict:
  all_rmse.append((key, results_dict[key][-1]))

all_rmse

In [ ]:
# get result of best model, which is BRNN GRU
gru_true_test_predicted = results_dict['GRU_True'][1]

In [ ]:
fig, axs = plt.subplots(3,2, figsize = (12,14))
n = 0
for i in range(3):
  for j in range(2):
    if n < len(colnames):
      axs[i,j].plot(test_raw[:,n], color = 'blue', label = 'Actual')
      axs[i,j].plot(gru_true_test_predicted[:,n], color = 'red', label = 'Predicted')
      axs[i,j].set_title(colnames[n])
      if n == len(colnames)-1:
        axs[i,j].set_ylabel('Number of shares')
      else:
        axs[i,j].set_ylabel('Price ($)')
      axs[i,j].set_xlabel('Time')
      axs[i,j].legend()
    n += 1
axs[-1,-1].axis('off')
plt.savefig('gru_true.png')
plt.show()

# Experiment 2

## Hyperparameter Tuning

In [ ]:
window_size = [20, 60]
batch_size = [32, 128]
learning_rate = [1e-2, 1e-3, 1e-4]
optimizer = [0,1,2]

configs = list(product(window_size, batch_size, learning_rate, optimizer))
np.random.seed(69)
np.random.shuffle(configs)

In [ ]:
data = []
nlayers, units = 2, 32 # more like 3
dropout_rate = 0.2
ntime = 1

for each_config in configs[1:]:
  print(ntime)
  tf.keras.backend.clear_session()
  window_size, batch_size, learning_rate, optimizer = each_config

  opt_name = ['SGD', 'RMSprop', 'Adam']
  print('Window size: {}, Batch size: {}, lr: {}, optimizer = {}'.format(window_size, batch_size,
                                                                         learning_rate, opt_name[optimizer]))

  X_train, y_train, X_val, y_val, X_test = prepare_data(train_scaled, val_scaled, test_scaled, window_size)

  model = Sequential()
  for i in range(nlayers):
    model.add(Bidirectional(GRU(units = units, return_sequences = True, input_shape = (window_size, nfeatures))))
    model.add(Dropout(dropout_rate))

  model.add(Bidirectional(GRU(units = units)))
  model.add(Dropout(dropout_rate)) 

  model.add(Dense(units = nfeatures))

  opt = SGD(learning_rate)
  if optimizer == 1:
    opt = RMSprop(learning_rate)
  elif optimizer == 2:
    opt = Adam(learning_rate = learning_rate)

  model.compile(optimizer = opt, loss = 'mean_squared_error')

  stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 20,
                                                restore_best_weights = True)
  
  history = model.fit(X_train, y_train, validation_data = (X_val, y_val), epochs = 200, 
                          batch_size = batch_size, verbose = 0, callbacks = [stop_early])

  plt.plot(history.history['loss'], label = 'train')
  plt.plot(history.history['val_loss'], label = 'validation')
  plt.legend()
  plt.show()
      
  predicted_val = model.predict(X_val)
  predicted_val = scaler.inverse_transform(predicted_val)
  for i in range(nfeatures):
    plt.plot(val_raw[:,i], label = 'actual')
    plt.plot(predicted_val[:,i], label = 'predicted')
    plt.legend()
    plt.show()

  rmse = mean_squared_error(val_raw, predicted_val, squared = False)
  print('RMSE: ', rmse)
  data.append(rmse)
  ntime += 1
  print()

In [ ]:
# Window size: 20, Batch size: 32, lr: 0.001, optimizer = Adam
best_window_size, best_batch_size, best_lr = 20, 32, 0.001
best_opt = Adam(best_lr)
nlayers, units, dropout_rate = 2, 32, 0.2

X_train, y_train, X_val, y_val, X_test = prepare_data(train_scaled, val_scaled, test_scaled, best_window_size)

# finding the best epoch

tf.keras.backend.clear_session()

model = Sequential()
for i in range(nlayers):
  model.add(Bidirectional(GRU(units = units, return_sequences = True, input_shape = (best_window_size, nfeatures))))
  model.add(Dropout(dropout_rate))

model.add(Bidirectional(GRU(units = units)))
model.add(Dropout(dropout_rate)) 

model.add(Dense(units = nfeatures))

model.compile(optimizer = best_opt, loss = 'mean_squared_error')

stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 20,
                                                restore_best_weights = True)
history = model.fit(X_train, y_train, validation_data = (X_val, y_val), epochs = 200, batch_size = best_batch_size, verbose = 0,
                    callbacks = [stop_early])



In [ ]:
val_loss = history.history['val_loss']
best_epoch = val_loss.index(min(val_loss)) + 1
best_epoch

## Using the best epoch and the hyperparameters found above and retrain model using just the train set

In [ ]:
# using the best epoch and the hyperparameters found above
# retrain model using just the train set

# Window size: 20, Batch size: 32, lr: 0.001, optimizer = Adam
predicted_ls = []
rmse_ls = []
X_train, y_train, X_val, y_val, X_test = prepare_data(train_scaled, val_scaled, test_scaled, best_window_size)

for ntime in range(5):
  # run 5 times
  tf.keras.backend.clear_session()

  model = Sequential()
  for i in range(nlayers):
    model.add(Bidirectional(GRU(units = units, return_sequences = True, input_shape = (best_window_size, nfeatures))))
    model.add(Dropout(dropout_rate))

  model.add(Bidirectional(GRU(units = units)))
  model.add(Dropout(dropout_rate)) 

  model.add(Dense(units = nfeatures))

  model.compile(optimizer = best_opt, loss = 'mean_squared_error')

  history = model.fit(X_train, y_train, epochs = best_epoch, batch_size = best_batch_size, verbose = 0)
        
  predicted = model.predict(X_test)
  predicted = scaler.inverse_transform(predicted)
  predicted_ls.append(predicted)

  rmse = mean_squared_error(test_raw, predicted, squared = False)
  rmse_ls.append(rmse)
  print(rmse)
  print()


In [ ]:
mean_predicted = np.mean(np.dstack(predicted_ls), axis = -1)

fig, axs = plt.subplots(3,2, figsize = (12,14))
n = 0
for i in range(3):
  for j in range(2):
    if n < len(colnames):
      axs[i,j].plot(test_raw[:,n], color = 'blue', label = 'Actual')
      axs[i,j].plot(mean_predicted[:,n], color = 'red', label = 'Predicted')
      axs[i,j].set_title(colnames[n])
      if n == len(colnames)-1:
        axs[i,j].set_ylabel('Number of shares')
      else:
        axs[i,j].set_ylabel('Price ($)')
      axs[i,j].set_xlabel('Time')
      axs[i,j].legend()
    n += 1
axs[-1,-1].axis('off')

plt.savefig('retrain_train.png')
plt.show()

In [ ]:
mean_rmse = np.mean(rmse_ls)
mean_rmse

## Using the best epoch and the hyperparameters found above and retrain model using the train and val set

In [ ]:
train_val_scaled = np.vstack((train_scaled, val_scaled))
full_test_set = np.vstack((train_val_scaled[-best_window_size:, :], test_scaled))

X_train_val = []
y_train_val = []
for i in range(len(train_val_scaled)-best_window_size):
  X_train_val.append(train_val_scaled[i:i+best_window_size, :])
  y_train_val.append(train_val_scaled[i+best_window_size,:])
X_train_val, y_train_val = np.array(X_train_val), np.array(y_train_val)
X_train_val.shape

X_test = []
for i in range(len(full_test_set)-best_window_size):
  X_test.append(full_test_set[i:i+best_window_size, :])
X_test = np.array(X_test)

In [ ]:

predicted_ls1 = []
rmse_ls1 = []

for ntime in range(5):
  tf.keras.backend.clear_session()

  model = Sequential()
  for i in range(nlayers):
    model.add(Bidirectional(GRU(units = units, return_sequences = True, input_shape = (best_window_size, nfeatures))))
    model.add(Dropout(dropout_rate))

  model.add(Bidirectional(GRU(units = units)))
  model.add(Dropout(dropout_rate)) 

  model.add(Dense(units = nfeatures))

  model.compile(optimizer = best_opt, loss = 'mean_squared_error')

  history = model.fit(X_train_val, y_train_val, epochs = best_epoch, batch_size = best_batch_size, verbose = 0)

  plt.plot(history.history['loss'], label = 'train')
  plt.legend()
  plt.show()
        
  predicted = model.predict(X_test)
  predicted = scaler.inverse_transform(predicted)
  predicted_ls1.append(predicted)

  rmse = mean_squared_error(test_raw, predicted, squared = False)
  rmse_ls1.append(rmse)
  print(rmse)
  print()


In [ ]:
mean_predicted1 = np.mean(np.dstack(predicted_ls1), axis = -1)

fig, axs = plt.subplots(3,2, figsize = (12,14))
n = 0
for i in range(3):
  for j in range(2):
    if n < len(colnames):
      axs[i,j].plot(test_raw[:,n], color = 'blue', label = 'Actual')
      axs[i,j].plot(mean_predicted1[:,n], color = 'red', label = 'Predicted')
      axs[i,j].set_title(colnames[n])
      if n == len(colnames)-1:
        axs[i,j].set_ylabel('Number of shares')
      else:
        axs[i,j].set_ylabel('Price ($)')
      axs[i,j].set_xlabel('Time')
      axs[i,j].legend()
    n += 1
axs[-1,-1].axis('off')

plt.savefig('both_train_val.png')
plt.show()

In [ ]:
mean_rmse1 = np.mean(rmse_ls1)
mean_rmse1